### Importing librairies

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import getpass
import pprint
import pandas as pd
from sqlalchemy import create_engine

### API connection

In [ ]:
c_id = getpass.getpass()
c_se = getpass.getpass()

In [ ]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id, client_secret=c_se, requests_timeout=1000))

Check if tracks_items_to_df() work with sevreal artists
Add genres
Add iteration throght pages


### Database connexion

In [ ]:
pw = "password1999"
connection_string = 'mysql+pymysql://root:' + pw + '@localhost:3306/'
engine = create_engine(connection_string, pool_pre_ping = False)

### Get maximum of track uri

Mise en forme des données : 2 tables : 
- une avec les infos du son (titre, allbum, artist name...)
- une avec les données de l'analyse
Les deux liés par l'URI

In [ ]:
def tracks_items_to_df(tracks_items) :
    result = pd.DataFrame(columns=["track_id","track_name","track_preview_url","track_popularity","track_duration","artists_id","artists_name","album_id","album_name","album_ release_date"])

    traks_fields = {"track_id":"id","track_name":"name","track_preview_url":"preview_url","track_duration":"duration_ms","track_explicit":"explicit","track_popularity":"popularity"}
    artist_fields = {"artist_id":"id","artist_name":"name"}
    album_fields = {"album_id":"id","album_name":"name","album_release_date":"release_date"}

    dicts_list = []

    for item in tracks_items :
        row = {} 
        for field in traks_fields.items() : 
            row[field[0]] = item[field[1]]

        for artist in item["artists"]: # Comment c'est géré quand il y a plusieurs artistes ?
            for field in artist_fields.items() : 
                row[field[0]] = artist[field[1]]

        for field in album_fields.items() : 
            row[field[0]] = item["album"][field[1]]
       
        dicts_list.append(row)

    return pd.DataFrame(dicts_list)
    

In [ ]:
genres = [  "acoustic",  "afrobeat",  "alt-rock",  "alternative",  "ambient",  "anime",  "black-metal",  "bluegrass",  "blues",  "bossanova",  "brazil",  "breakbeat",  "british",  "cantopop",  "chicago-house",  "children",  "chill",  "classical",  "club",  "comedy",  "country",  "dance",  "dancehall",  "death-metal",  "deep-house",  "detroit-techno",  "disco",  "disney",  "drum-and-bass",  "dub",  "dubstep",  "edm",  "electro",  "electronic",  "emo",  "folk",  "forro",  "french",  "funk",  "garage",  "german",  "gospel",  "goth",  "grindcore",  "groove",  "grunge",  "guitar",  "happy",  "hard-rock",  "hardcore",  "hardstyle",  "heavy-metal",  "hip-hop",  "holidays",  "honky-tonk",  "house",  "idm",  "indian",  "indie",  "indie-pop",  "industrial",  "iranian",  "j-dance",  "j-idol",  "j-pop",  "j-rock",  "jazz",  "k-pop",  "kids",  "latin",  "latino",  "malay",  "mandopop",  "metal",  "metal-misc",  "metalcore",  "minimal-techno",  "movies",  "mpb",  "new-age",  "new-release",  "opera",  "pagode",  "party",  "philippines-opm",  "piano",  "pop",  "pop-film",  "post-dubstep",  "power-pop",  "progressive-house",  "psych-rock",  "punk",  "punk-rock",  "r-n-b",  "rainy-day",  "reggae",  "reggaeton",  "road-trip",  "rock",  "rock-n-roll",  "rockabilly",  "romance",  "sad",  "salsa",  "samba",  "sertanejo",  "show-tunes",  "singer-songwriter",  "ska",  "sleep",  "songwriter",  "soul",  "soundtracks",  "spanish",  "study",  "summer",  "swedish",  "synth-pop",  "tango",  "techno",  "trance",  "trip-hop",  "turkish",  "work-out",  "world-music"]

In [ ]:
def get_df_from_genres(genres) :

    df = pd.DataFrame(columns=["track_id","track_name","track_genre","track_preview_url","track_popularity","track_duration","artist_id","artist_name","album_id","album_name","album_release_date"])
    for genre in genres :
        df_genre = pd.DataFrame(columns=["track_id","track_name","track_genre","track_preview_url","track_popularity","track_duration","artist_id","artist_name","album_id","album_name","album_release_date"])
        for offset in range(0,1000,50) :

            query = "genre:{}".format(genre)
            df_offset = sp.search(q=query, type='track', limit=50, offset = offset)["tracks"]["items"]
            df_offset = tracks_items_to_df(df_offset)
            try : 
                df_offset.insert(2,"track_genre",genre)
            except : # if it's fail to get specific genre
                print(genre) 
                break

            df_genre = pd.concat([df_genre,df_offset])

        df_genre.to_sql("df_{}".format(genre.replace("-","")), connection_string, schema='spotify', if_exists='replace', index = False)
        
        df = pd.concat([df,df_genre])

    return df

In [ ]:
df = get_df_from_genres(genres)

Failed to get genres :  Bossanova, disney, holidays,metal-misc, movies, new-release, philippines-opm, post-dubstep, rainy-day, road-trip, soundtracks, summer, work-out

In [ ]:
df.to_sql("df_all_genres", connection_string, schema='spotify', if_exists='replace', index = False)

In [ ]:
df.shape

### Merging tables to df_all_genres

In [ ]:
tables_names = ["df_{}".format(genre.replace("-","")) for genre in genres]

In [ ]:
df_all_genres = pd.read_sql("SELECT * FROM `spotify`.`{}`".format(tables_names[0]), engine)

for table in tables_names[1:] :
    MYSQL_CONNECTION_STRING = "mysql://root@localhost/ironhack"
    df_genre = pd.read_sql("SELECT * FROM spotify.{}".format(table), engine)
    df_all_genres = pd.concat([df_all_genres,df_genre])

df_all_genres

### Get more tracks with recommandations

In [ ]:
def recommend(artist,track,genre):
    rec_songs = {}
    rec = sp.recommendations(seed_artists=artist,seed_tracks=track,seed_genres=genre, country='US', limit=10)
    return rec["tracks"]

In [ ]:
def get_rec(df):
    df_1 = pd.DataFrame()
    for i in range(len(df)):
        r = sp.recommendations(seed_artists=[str(df['artist_id'][i])],seed_tracks=[str(df['track_id'][i])],seed_genres=['pop'],  country='US',limit=100)
        df_2 = tracks_items_to_df(r['tracks'])
        df_1 = pd.concat([df_1,df_2]).reset_index(drop=True)
    return df_1

In [ ]:
rec = recommend(artist=['7dGJo4pcD2V6oG8kP0tJRR','4Cdvtu5XUmCKPwcnue11xS'],track=['1v7L65Lzy0j0vdpRjJewt1','3BzfkAB5qE3BNiVsdA35nu'],genre=['hip-hop'])
tracks_items_to_df(rec)

### Get audio feautures from tracks URI

In [ ]:
# get audio features, et audio analysis
sp.audio_analysis("spotify:track:2QjOHCTQ1Jl3zawyYOpxh6") #Too much
sp.audio_features("spotify:track:2QjOHCTQ1Jl3zawyYOpxh6")[0].keys() #Ce qu'on va utiliser

In [ ]:
# for style in musical_styles : 
#     query = "genre:'{}'".format(style)
#     search_results = sp.search(q=query, type='track', limit=50)  # Limitez le nombre de résultats ici